*tysserand* includes functions to help using [napari](https://napari.org/) for interactive image and network visualization and annotation.  
Thanks to napari you can zoom, pan and hide or change the intensity of individual channels.  
We define the ground truth of network of the multiplex Immuno Fluorescence image.  
We start with a Delaunay reconstruction, then we manually add and delete some edges.  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
from tysserand import tysserand as ty
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000 


# Interactive networks visualization and annotation

In [ ]:
# where we save and load annotations
data_dir = Path("/home/mouneem/Projects/tysserand/tysserand/new_data/")

## Delaunay triangulation

In [ ]:
nodes = pd.read_csv(data_dir / 'min15T011146_80032.fcs.csv', usecols=['x','y', 'class'])
print(len(nodes))
#nodes = nodes.sample(n = round(len(nodes) / 2) )
print(len(nodes))


In [ ]:
## Adjust
img = plt.imread(data_dir / "4518.tiff")

from PIL import Image, ImageOps
#img = ImageOps.grayscale(img)

coords = nodes.loc[:,['x','y']].values

class_colors = ['#F85446', '#813FE0', '#52DEF7', '#62E03F', '#FFCE36', '#FA6C17',
               '#D805E3', '#0155FA', '#0BE35B', '#FFF117', '#DDFF7D', '#78E3C8']
classes = list(nodes['class'].unique())
print(classes)
dico_col = {classes[0]:class_colors[0],
            classes[1]:class_colors[1],
            classes[2]:class_colors[2],
            classes[3]:class_colors[3],
            classes[4]:class_colors[4],
            classes[5]:class_colors[5],
            classes[6]:class_colors[6],
           }
colors = []
for cl in nodes['class']:
    colors.append(dico_col[cl])
    

    
    
min_x = min(coords[:, 0])
min_y = min(coords[:, 1])

max_x = max(coords[:, 0])
max_y = max(coords[:, 1])


print(min_x, min_y)

import cv2
#img = cv2.flip(img, 0)



We first load existing nodes annotations. For each cell type we define a node color, then we define a first version of edges with the delaunay triangulation method.

In [ ]:
import numpy

coords2 = coords


#coords2[:, 0] = coords2[:, 0] - 5600
#coords2[:, 1] = coords2[:, 1] - 18500

coords_x_diff = max(coords2[:, 0]) - min(coords2[:, 0])
coords_y_diff = max(coords2[:, 1]) - min(coords2[:, 1])

print(coords_x_diff , coords_y_diff)

img2 = img
#img2 = cv2.resize(img, (coords_x_diff , coords_y_diff)) 
[h, w] = img.shape
ratio_x = h / coords_x_diff
ratio_y = w / coords_y_diff
#print(ratio_x, ratio_y)

In [ ]:
fig, ax = ty.showim(img2, figsize=(150, 150))

ratio = 1
ratio = 0.1

plt.grid()

#pairs = ty.build_delaunay(coords2)
#distances = ty.distance_neighbors(coords2, pairs)
col_nodes = colors
ax.scatter(coords2[:,0] * ratio, coords2[:,1] * ratio, c=col_nodes,  zorder=10, s = 1)




In [ ]:
coords = coords2
fig, ax = ty.showim(img2, figsize=(200, 200))


pairs = ty.build_delaunay(coords)
distances = ty.distance_neighbors(coords, pairs)

    
# superimpose network to mIF image
ty.plot_network(coords, pairs, col_nodes=colors, col_edges='w', ax=ax)


## Napari-assisted annotations

We use napari to hide channels or tweak their intensity. In the viewer we can add, move or delete points and edges that will be considered as nodes by *tysserand*. These data are stored as a `dictionnary` of annotations.

In [ ]:
import napari

We load napari with independent layers for each channel:

In [ ]:
viewer = napari.Viewer()
ty.visualize(viewer, img, colormaps='rgb')

We add nodes positions as annotations:

nodes_coords: nb_nodes x 2 array  
edges_coords: list of 2 x 2 arrays of length nb_edges

In [ ]:
# make colors for nodes
#                 orange      blue      green
    
annotations = ty.make_annotation_dict(
    coords, pairs=pairs,
    nodes_class=nodes['marker'],
    nodes_class_color_mapper=dico_col,
)

In [ ]:
ty.add_annotations(viewer, annotations)


Now you can inspect annotations and manually remove, modify or add nodes and edges.  
Select the right layer first (nodes or edges), and if you want to add nodes, select the apropriate `face color` that will be considered as a cell type indication. To do so select first an existing node with the desired color and copy the color code before creating a new node.

If you have added edges, to match their manually defined positions to their respective nodes and exact nodes positions *tysserand* provides an utility:

In [ ]:
# get annotation layers names
layer_nodes_name, layer_edges_name = ty.get_annotation_names(viewer)
# get modified annotations
annotations = ty.get_annotation_dict(viewer, layer_nodes_name, layer_edges_name)
# clean edges positions
new_edges, pairs = ty.assign_nodes_to_edges(annotations['nodes_coords'], annotations['edges_coords'])
annotations['edges_coords'] = new_edges
annotations['pairs'] = pairs

You can now display the corrected edges with:

In [ ]:
ty.update_edges(viewer, annotations)

You can finally save annotations with:

In [ ]:
ty.save_annotations(data_dir / 'annotations.pkl', viewer=viewer)

If you want to reuse annotations:

In [ ]:
annotations = ty.load_annotations(data_dir / 'annotations.pkl')

viewer = napari.Viewer()
ty.visualize(viewer, img, colormaps='rgb')
ty.add_annotations(viewer, annotations)

# 